# Транспортная Задача
## Вариант № 10

In [1]:
import numpy as np
from scipy.optimize import linprog
import warnings

Метод северо-запада

In [2]:
def sev_zap(a_, b_, c_):
    a = np.copy(a_)
    b = np.copy(b_)
    c = np.copy(c_)

    if a.sum() > b.sum():
        b = np.hstack((b, [a.sum() - b.sum()]))
        c = np.hstack((c, np.zeros(len(a)).reshape(-1, 1)))
    elif a.sum() < b.sum():
        a = np.hstack((a, [b.sum() - a.sum()]))
        c = np.vstack((c, np.zeros(len(b))))

    m = len(a)
    n = len(b)
    i = 0
    j = 0
    funk = 0
    x = np.zeros((m, n), dtype=int)

    while (i < m) and (j < n):
        x_ij = min(a[i], b[j])
        funk += c[i, j] * min(a[i], b[j])
        a[i] -= x_ij
        b[j] -= x_ij
        x[i, j] = x_ij

        if a[i] > b[j]:
            j += 1
        elif a[i] < b[j]:
            i += 1
        else:
            i += 1
            j += 1

    return x, funk


In [3]:
def find_coordinates(c_min):
    c = np.inf

    for i in range(c_min.shape[0]):
        for j in range(c_min.shape[1]):
            if (c_min[i, j] != 0) and (c_min[i, j] < c):
                c = c_min[i, j]
                i_, j_ = i, j

    return i_, j_


def min_elem(a_, b_, c_):
    a = np.copy(a_)
    b = np.copy(list(reversed(b_)))
    c = np.copy(c_)

    if a.sum() > b.sum():
        b = np.hstack((b, [a.sum() - b.sum()]))
        c = np.hstack((c, np.zeros(len(a)).reshape(-1, 1)))
    elif a.sum() < b.sum():
        a = np.hstack((a, [b.sum() - a.sum()]))
        c = np.vstack((c, np.zeros(len(b))))

    m = len(a)
    n = len(b)
    x = np.zeros((m, n), dtype=int)
    funk = 0

    while 1:
        c_min = np.zeros((m, n))

        for i in range(m):
            for j in range(n):
                c_min[i, j] = (c[i, j] * min(a[i], b[j]))

        i, j = find_coordinates(c_min)
        x_ij = int(min(a[i], b[j]))
        x[i, j] = x_ij
        funk += int(c_min[i, j])
        a[i] -= x_ij
        b[j] -= x_ij

        if len(c_min[c_min > 0]) == 1:
            break

    return x, funk


def prepare(a, b):
    m = len(a)
    n = len(b)
    h = np.diag(np.ones(n))
    v = np.zeros((m, n))
    v[0] = 1

    for i in range(1, m):
        h = np.hstack((h, np.diag(np.ones(n))))
        k = np.zeros((m, n))
        k[i] = 1
        v = np.hstack((v, k))

    return np.vstack((h, v)).astype(int), np.hstack((b, a))


def potenz(a_, b_, c_):
    a = np.copy(a_)
    b = np.copy(b_)
    c = np.copy(c_)

    if a.sum() > b.sum():
        b = np.hstack((b, [a.sum() - b.sum()]))
        c = np.hstack((c, np.zeros(len(a)).reshape(-1, 1)))
    elif a.sum() < b.sum():
        a = np.hstack((a, [b.sum() - a.sum()]))
        c = np.vstack((c, np.zeros(len(b))))

    m = len(a)
    n = len(b)
    A_eq, b_eq = prepare(a, b)
    res = linprog(c.reshape(1, -1), A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='simplex')

    return res.x.astype(int).reshape(m, n), res.fun.astype(int)


In [4]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore')

    a = np.array([15, 10, 35]) # Запас

    b = np.array([11, 11, 11, 16, 11]) # Потребности

    D = np.array([[3, 4, 5, 15, 24],
                  [19, 2, 22, 4, 13],
                  [20, 27, 1, 17, 19]])

    x, funk = sev_zap(a, b, D)
    print("\n#########################################################################\n")
    print('Метод Cеверо-Pападного угла: \n', x)
    print('\nЦелевая функция: \n> ', funk)
    print("\n#########################################################################\n")

    x1, funk1 = min_elem(a, b, D)
    print('Метод минимального элемента: \n', x1)
    print('\nЦелевая функция: \n> ', funk1)
    print("\n#########################################################################\n")

    x2, funk2 = potenz(a, b, D)
    print('Метод потенциалов: \n', x2)
    print('\nЦелевая функция: \n> ', funk2)
    print("\n#########################################################################\n")




#########################################################################

Метод Cеверо-Pападного угла: 
 [[11  4  0  0  0]
 [ 0  7  3  0  0]
 [ 0  0  8 16 11]]

Целевая функция: 
>  618

#########################################################################

Метод минимального элемента: 
 [[ 9  6  0  0  0]
 [ 0 10  0  0  0]
 [ 2  0 11 11 11]]

Целевая функция: 
>  518

#########################################################################

Метод потенциалов: 
 [[11  4  0  0  0]
 [ 0  7  0  3  0]
 [ 0  0 11 13 11]]

Целевая функция: 
>  516

#########################################################################

